In [ ]:
BASE_DIR = "/content/drive/MyDrive/ai-video-generator"

In [ ]:
!pip install transformers torch gtts moviepy Pillow python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import os

ASSETS_DIR = os.path.join(BASE_DIR, "assets")
IMAGES_DIR = os.path.join(ASSETS_DIR, "images")
AUDIO_DIR = os.path.join(ASSETS_DIR, "audio")
OUTPUT_DIR = os.path.join(ASSETS_DIR, "output")
HF_MODEL_CACHE = os.path.join(BASE_DIR, "hf_models")

os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.environ['TRANSFORMERS_CACHE'] = HF_MODEL_CACHE


In [ ]:
from transformers import pipeline, set_seed
from gtts import gTTS
from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips
from PIL import Image
import requests
from io import BytesIO

generator = pipeline("text-generation", model="distilgpt2", framework="pt")
set_seed(42)

# def generate_script(topic):
#     prompt = f"Write a short and engaging video script about: {topic}. Keep it under 150 words."
#     result = generator(prompt, max_length=150, num_return_sequences=1)
#     return result[0]['generated_text']

def generate_script(topic):
    prompt = f"Write a short and engaging video script about: {topic}. Keep it under 150 words."
    result = generator(prompt, max_length=150, num_return_sequences=1)
    script = result[0]['generated_text']
    word_count = len(script.split())
    return script, word_count

def fetch_image_urls(query, count=3):
    PIXABAY_API_KEY = "YOUR_KEY_HERE" #Images
    url = f"https://pixabay.com/api/?key={PIXABAY_API_KEY}&q={query}&image_type=photo&per_page={count}"
    response = requests.get(url).json()
    return [hit['webformatURL'] for hit in response['hits']]

def download_images(image_urls):
    paths = []
    for i, url in enumerate(image_urls):
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        path = os.path.join(IMAGES_DIR, f"img{i}.jpg")
        img.save(path)
        paths.append(path)
    return paths

def generate_audio(text, filename="narration.mp3"):
    tts = gTTS(text)
    audio_path = os.path.join(AUDIO_DIR, filename)
    tts.save(audio_path)
    return audio_path

def create_video(image_paths, audio_path, output_name="final_video.mp4"):
    from moviepy.editor import AudioFileClip, ImageClip, concatenate_videoclips

    # Load the audio to get its duration
    audio = AudioFileClip(audio_path)
    total_duration = audio.duration
    num_images = len(image_paths)

    # Calculate duration per image to evenly span the entire audio
    duration_per_image = total_duration / num_images

    # Create image clips with adjusted durations
    clips = [ImageClip(path).set_duration(duration_per_image) for path in image_paths]

    # Concatenate image clips and attach audio
    video = concatenate_videoclips(clips, method="compose")
    video = video.set_audio(audio).set_duration(audio.duration)

    # Export final video
    video_path = os.path.join(OUTPUT_DIR, output_name)
    video.write_videofile(video_path, fps=24)
    return video_path



/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
  if event.key is 'enter':

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# topic = "space exploration"  # Or use input("Enter a video topic: ")
# script = generate_script(topic)
# print("Script:\n", script)

# image_urls = fetch_image_urls(topic)
# image_paths = download_images(image_urls)
# audio_path = generate_audio(script)
# video_path = create_video(image_paths, audio_path)
# print("Video saved at:", video_path)
# 📌 Topic
topic = "space exploration"  # or use input("Enter a topic: ")

# 📜 Generate script and count words
script, word_count = generate_script(topic)
print("Generated Script:\n", script)
print(f"Word count: {word_count}")

# 📷 Decide number of images (1 per 25 words, min 3, max 15)
num_images = min(max(word_count // 25, 3), 15)
print(f"\nFetching {num_images} images based on script length...")

# 🔽 Fetch images
image_urls = fetch_image_urls(topic, count=num_images)
image_paths = download_images(image_urls)

# 🎙 Generate audio narration
audio_path = generate_audio(script)

# 🎞 Create synced video
video_path = create_video(image_paths, audio_path)
print("\n🎬 Video created at:", video_path)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Script:
 Write a short and engaging video script about: space exploration. Keep it under 150 words. It's really simple:
This is a great and exciting program, thanks to all who have invested in the video project (and will always be supported for the duration of time) and its continued work.
I appreciate your support of the program and I hope to see you out there getting something out of the way. I like to get to know you before you're done:
What you're seeing in the video series and the community is awesome. I hope all the players in the program are here to help.
You've been a part of this project since August 2009 in the form of the following blogs:
The New Earth, and the Creation
Word count: 127

Fetching 5 images based on script length...
Moviepy - Building video /content/drive/MyDrive/ai-video-generator/assets/output/final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/ai-video-generator/assets/output/final_video.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/ai-video-generator/assets/output/final_video.mp4

🎬 Video created at: /content/drive/MyDrive/ai-video-generator/assets/output/final_video.mp4


In [ ]:
video_path = "/content/drive/MyDrive/ai-video-generator/assets/output/final_video.mp4"

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def play_video(path):
    mp4 = open(path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""
      <video width=600 controls>
            <source src="{data_url}" type="video/mp4">
      </video>
    """)

play_video(video_path)
